# Exploring trends in UK academia using PhD thesis metadata
#### Markus Hauru, January 2020

Since 2009 the British Library has been operating its [Electronic Theses Online Service](https://ethos.bl.uk) or EThOS service, that keeps track of PhD theses accepted at UK higher education institutions. The Library also makes public a [dataset](https://data.bl.uk/ethos/) of metadata for all theses in their catalogue. Quoting the EThOS website,

>The EThOS dataset lists virtually all UK doctoral theses ever awarded, some 500,000 dating back to 1787. All UK HE institutions are included, but we estimate records are missing for around 10,000 titles (2%).

The latest version of the dataset is from March 2018. The data includes title, year, author, and institution for each thesis, as well as a link to a full record of each thesis, which may or may not include things like keywords or access to full texts, depending on the thesis.

In this notebook I explore this dataset to identify historical trends in UK academia or other interesting features it may display. While an interesting project in itself, this notebook is also an exercise for myself, to learn some basic exploratory data science tools and techniques, like basics of pandas, some network analysis, and new visualization te.

## Setup

First off, some imports of python libraries we'll be needing, and loading the data file into a pandas DataFrame. The script will automatically download the data file into the current working directory if it isn't there yet.

In [54]:
import os
import operator as opr
import scipy as sp
import numpy as np
import pandas as pd
import networkx as nx
import community  # Network community finding
from matplotlib import pyplot as plt
from matplotlib import cm  # color maps
import seaborn as sns
# for pretty-printing in notebooks
from IPython.display import display, HTML

In [12]:
# For pretty and interactive matplotlib plots in Jupyter Lab.
%matplotlib widget

In [13]:
datafile = "EThOSCSV_201803.csv"
if not os.path.isfile(datafile):
    # Download and/or unzip the data file from the EThOS website.
    # We need a couple more imports for this.
    import zipfile

    datazip = "EThOSCSV_201803.zip"
    if not os.path.isfile(datazip):
        import requests

        dataurl = "https://data.bl.uk/ethos/EThOSCSV201803.zip"
        with requests.Session() as s:
            headers = {
                "User-agent": "Mozilla/5.0 (X11; Linux i686; rv:64.0) Gecko/20100101 Firefox/64.0"
            }
            r = s.get(dataurl, headers=headers)
            with open(datazip, "wb") as f:
                f.write(r.content)
    with zipfile.ZipFile(datazip, "r") as z:
        z.extractall(".")

In [14]:
df = pd.read_csv(datafile, encoding="ISO-8859-2")

To get an idea of what the rows in the DataFrame look like, here are some samples:

In [15]:
df.head()

,Title,Author,Institution,Date,Qualification,EThOS URL
0,Computation and measurement of turbulent flow ...,"Loizou, Panos A.",University of Manchester,1989.0,Thesis (Ph.D.),http://ethos.bl.uk/OrderDetails.do?uin=uk.bl.e...
1,Prolactin and growth hormone secretion in norm...,"Prescott, R. W. G.",University of Newcastle upon Tyne,1983.0,Thesis (Ph.D.),http://ethos.bl.uk/OrderDetails.do?uin=uk.bl.e...
2,Influence of strain fields on flame propagation,"Mendes-Lopes, J. M. C.",University of Cambridge,1983.0,Thesis (Ph.D.),http://ethos.bl.uk/OrderDetails.do?uin=uk.bl.e...
3,"Connectivity, flow and transport in network mo...","Robinson, Peter Clive",University of Oxford,1984.0,Thesis (Ph.D.),http://ethos.bl.uk/OrderDetails.do?uin=uk.bl.e...
4,The theory and implementation of a high qualit...,"Lower, K. N.",University of Bristol,1985.0,Thesis (Ph.D.),http://ethos.bl.uk/OrderDetails.do?uin=uk.bl.e...


The first observation to make is that the data is remarkably clean. There are a few NaNs that we need to drop:

In [16]:
print("Number of rows with NaNs: {}".format(df.isnull().any(axis=1).sum()))
df = df.dropna()

Number of rows with NaNs: 13


but with that done, there's little else to do. The Qualification column holds some oddities and misunderstandings, and there are few suspiciously short titles (my favourite being the 1977 thesis called "Beds"), but other than that, things seem in order. Most notably, the Institution and Date fields, which we'll be getting a lot of mileage from, seem spotless, without even a single typoed university name.

What is of some concern are the 10,000 or so theses that the British Library estimates are missing from the data set. Nowadays that's about a year's worth of theses, but it's also roughly the total number of theses in the data from before 1956. Now clearly some of the missing ones are from the last few years: Even though the set is from 2018, the number of theses in the database drops sharply after 2015, even though it's been climbing up to that point. But there's probably a significant number missing from the early years as well, considering issues of record keeping. To illustrate the late and early parts of the data set:

In [17]:
thesiscounts_by_year = df.Date.value_counts().sort_index()

In [18]:
fig = plt.figure(figsize=(7, 3))
ax1 = fig.add_subplot(1, 2, 1)
ax1.plot(thesiscounts_by_year[2005:])
plt.ylabel("Number of PhD theses")
plt.xlabel("Year")

ax2 = fig.add_subplot(1, 2, 2)
ax2.plot(thesiscounts_by_year[:1940])

plt.tight_layout();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Before the end of WW1 the sample sizes for each year are clearly quite low, and based on this, as well as some odd features of the data pre-WW1, I've chosen to limit the time period to study to 1925-2015.

In [19]:
late_cutoff = 2015
early_cutoff = 1925
thesiscounts_by_year = thesiscounts_by_year[early_cutoff:late_cutoff]
df = df[(early_cutoff <= df.Date) & (df.Date <= late_cutoff)]

I also drop the columns we won't be using for anything. Makes for nicer reading when we print out slices of the DataFrame

In [20]:
df.drop(["Qualification", "EThOS URL"], axis=1, inplace=True)

## Number of theses per year and institution

With that basic setup out of the way, let's get into this.

First, a basic look at the number of PhD theses accepted in the UK over our chosen time period.

In [22]:
plt.figure()
plt.semilogy(thesiscounts_by_year)
plt.ylabel("Number of PhD theses.")
plt.xlabel("Year");

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Note the logarithmic vertical axis. The two word summary here would clearly be "exponential growth". In 1925 a bit less than a hundred PhDs were awarded, and now we are approaching 20 000 per year. The most notable features are
* a huge dip during WW2 in the middle of the otherwise steady exponential period from 1925 to 1960. (In the early-times plot a couple of cells above you can also see a dip during WW1, although the signal there is more noisy.) There's a corresponding bump a few years after WW2, presumably from people who postponed the start of their studies until after the war and are then graduating in large batches.
* a period of even faster exponential growth in the 1960-1980 window. One explanation for this is baby boomers hitting typical PhD age (compare the above to a UK birth rate plot [here](https://www.ons.gov.uk/peoplepopulationandcommunity/birthsdeathsandmarriages/livebirths/bulletins/birthsummarytablesenglandandwales/2017) and shift by 25 or so years), but probably the most important driver of this growth is the huge structural change in academia that we'll see in a moment.
* a more moderate but steady pace from roughly 1980 onwards, that may be slowing down lately.

Note that when I talk about rate or pace of growth here I'm talking in multiplicative terms, meaning for instance that since 1980 the number of new PhDs has been growing by a roughly constant _percentage_ per year, not a constant number.

Next let's look at the contributions of different institutions to the total output of PhDs. The below plot shows the total thesis count per year divided between different universities.

In [23]:
thesiscounts_by_yearandinst = (
    df.groupby(["Institution", "Date"])
    .size()
    .unstack("Institution")
    .fillna(0.0)
)
# Sort institutions based on PhD output in the last year of the time window.
thesiscounts_by_yearandinst = thesiscounts_by_yearandinst.sort_values(
    late_cutoff, axis="columns", ascending=False
)
instratios_by_year = thesiscounts_by_yearandinst.divide(
    thesiscounts_by_year, axis="index"
)

In [24]:
# We can't show the full legend with dozens of entries, so we restrict to a few of the
# most important ones.
num_legendentries = 10

# We use one of matplotlibs color maps with 20 colors since we have a lot areas to plot,
# but to not have similar colors right next to each other we mix up the order.
colors = cm.tab20b.colors
num_colors = len(colors)
colors = [colors[i * 3 % num_colors] for i in range(num_colors)]

plt.figure(figsize=(10, 5))
lines = plt.stackplot(
    instratios_by_year.index, instratios_by_year.values.T, colors=colors
)
plt.ylabel("Ratio of PhDs by institution\n to total number of PhDs that year")
plt.xlabel("Year")
plt.ylim(0.0, 1.0)
plt.xlim(early_cutoff, late_cutoff)
# Some fancy formatting of the legend. We show the most important legend entries in the same order
# that they appear in in the figure and position the legend out of the way.
plt.legend(
    reversed(lines[:num_legendentries]),
    reversed(instratios_by_year.columns[:num_legendentries]),
    loc="lower left",
    bbox_to_anchor=(1.0, 0.0),
)
plt.tight_layout();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

The legend names a few of the most important instutions, starting from the bottom of the plot.

This shows the drastic change in the structure of higher education between roughly 1960 and 1980, that I mentioned above: There's an explosion in the number of PhDs coming from small universities. Until 1960 or so Oxford, Cambridge and Edinburgh produced 70-80% of PhDs in the country. By 1980 it was around to 20%, and has stayed roughly constant since. They remain the largest PhD factories, together with Manchester and Imperial College London, but the field has been filled with dozens of smaller institutions. During the shift many new universities were founded (Sussex 1961, York 1963, Warwick 1965, etc.), many others expanded greatly, and various colleges and other educational institutions were turned into universities. I'm not a historian, but for some background and context, check Wikipedia on for instance [University Grant Comission](https://en.wikipedia.org/wiki/University_Grants_Committee_(United_Kingdom)), [Robbins Report](https://en.wikipedia.org/wiki/Robbins_Report), and [Education Act of 1962](https://en.wikipedia.org/wiki/Education_Act_1962). I was previously aware that there was an expansion of higher education in many Western countries around this time, but I have to say that the magnitude and speed of the shift in the above plot surprised me.

There are also some interesting individual stories in this same plot. Oxford used to dominate its rival Cambridge with a doctoral output several times larger until the late 60s, after which the two have been roughly equal. Even Oxford was for a time ecplised by Edinburgh though, which in some years produced more than 40% of all PhDs in the UK. (It should be said though that I suspect that these early years may suffer from a significant number of missing theses, that may skew the data to some degree.) Imperial has an interesting bulge of activity during the 1960s as it went through [a rapid expansion](https://en.wikipedia.org/wiki/Imperial_College_London#20th_century), a bit before many others followed suit. Overall, this millenium there's been again a slight squeeze, where the big institutions have been growing and the smaller ones have been shrinking, although these changes are very minor compared to the opposite explosion in the 60s and 70s.

Just to underline the fact that the fall in the relative importance of the top few institutions is not at all due to shrinking on their part but merely the growth of others, here are the absolute numbers of PhDs produced by the 8 institutions that contributed the most in 1960.

In [25]:
num_insts = 8
counts_top = thesiscounts_by_yearandinst.sort_values(
    1960, axis=1, ascending=False
)
plt.figure(figsize=(10, 5))
lines = plt.plot(counts_top.iloc[:, :num_insts])
plt.ylabel("Number of PhDs produced")
plt.xlabel("Year")
plt.legend(
    lines,
    counts_top.columns[:num_insts],
    loc="lower left",
    bbox_to_anchor=(1.0, 0.0),
)
plt.tight_layout();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Words, words, words...

So far we've only looked at where and when PhD theses have been written. The dataset doesn't hold that much more information to toy with. It doesn't list departments or faculties, keywords or research fields, nor do I have easy access to full texts of the theses. But what it does have is the titles of these theses. Let's see what we can learn from them.

### Setting up the title analysis

Before we get to looking at trends in titles, we need to spend a moment setting up some machinery that we can then milk.

Now thesis titles aren't exactly prose. They typically aren't full sentences and their grammatical forms vary wildly, which makes analysis of their linguistic structure tricky and, I would guess, in many cases a bit futile. Unlike with more structured text, almost as much information as in the title itself is contained in just a list of words appearing in the title. Lists of words are also easy to analyse, so we'll go with that.

To start, we strip the titles of any punctation and so called stop words like articles and prepositions that don't tell us much, and make everything lower case. We could also do what's called stemming, and collapse for instance "study", "studies", "studied" and "studying" all into a single word. I choose not to do this because as we'll see below, different inflections of the same stem word sometimes appear in significantly different kinds of titles. Moreover, with academic vocabulary, doing this properly isn't straight-forward: the stemming algorithm should for instance know to combine "phenomenon" and "phenomena", but perhaps not "phenomenal" or "phenomenology", and certainly not "phenolic". A quick experiment that I did with one stemmer also suggested that the following results wouldn't be greatly affected.

Another choice I make is to remove hyphens as unnecessary punctuation. Here you lose some and you win some. "Post-modern" should certainly be grouped together with "postmodern", but "biodiversity-ecosystem" (which appears in 5 titles) would be better split into two words.

In [26]:
def splitstr(s):
    """ For a string, remove most punctuation, lower case, and split into words. Return the words.
    """
    puncts = "!\"&'(),./:;<=>?[\\]`{|}-"
    return s.lower().translate(str.maketrans("", "", puncts)).split()


def filter_stopwords(l):
    """ Take a list of strings, filter out prepositions, articles, and other stop words.
    We use a list of English stop words found here: https://www.textfixer.com/tutorials/common-english-words.txt
    """
    # We stop black from automatically splitting this across a gazillion lines with the fmt: off/on.
    # fmt: off
    stops = ["a", "able", "about", "across", "after", "all", "almost", "also", "am", "among", "an", "and", "any", "are", "as", "at", "be", "because", "been", "but", "by", "can", "cannot", "could", "dear", "did", "do", "does", "either", "else", "ever", "every", "for", "from", "get", "got", "had", "has", "have", "he", "her", "hers", "him", "his", "how", "however", "i", "if", "in", "into", "is", "it", "its", "just", "least", "let", "like", "likely", "may", "me", "might", "most", "must", "my", "neither", "no", "nor", "not", "of", "off", "often", "on", "only", "or", "other", "our", "own", "rather", "said", "say", "says", "she", "should", "since", "so", "some", "than", "that", "the", "their", "them", "then", "there", "these", "they", "this", "tis", "to", "too", "twas", "us", "wants", "was", "we", "were", "what", "when", "where", "which", "while", "who", "whom", "why", "will", "with", "would", "yet", "you", "your"]
    # fmt: on
    return tuple(s for s in l if s not in stops)

In [27]:
df["Words"] = df["Title"].apply(splitstr).apply(filter_stopwords)

A handy thing to have: A function that, given a word, spits out example titles where it appears.

In [28]:
def get_example_titles(word, n=1):
    titles_with_word = df[df["Words"].apply(lambda x: word in x)]["Title"]
    n = min(n, len(titles_with_word))
    examples = tuple(titles_with_word.sample(n).values)
    return examples

To get the title analysis going, we want three different objects:
* A DataFrame that lists the total number of titles in which each word appears.
* The same thing but now per year. (We could do per institution as well, but let's leave that for later.)
* A so called co-occurrence graph (or network), i.e. a weighted graph where nodes are words and edges tell us which words appear together in titles and how often.

In [29]:
wordcounts_by_year = (
    df.groupby("Date")["Words"]
    .apply(lambda x: pd.Series(np.concatenate(x.tolist())).value_counts())
    .unstack("Date")
    .fillna(0.0)
    .T
)

In [30]:
total_wordcounts = wordcounts_by_year.sum(axis=0)
# Sort both total_wordcounts and wordcounts_by_year to have the most common words first.
order = (-total_wordcounts).argsort()
total_wordcounts = total_wordcounts[order]
allwords = total_wordcounts.index
wordcounts_by_year = wordcounts_by_year.reindex(allwords, axis=1)

Let's also compute the occurrences of each word per a 1000 thesis, in each given year, to measure the relative popularity.

In [31]:
wordratios_by_year = 1000*wordcounts_by_year.divide(thesiscounts_by_year, axis=0)

Making the co-occurrence graph requires some thought. First off, we've got around 190,000 distinct words in about 450,000 titles (academics like their jargon). However, only 20,000 or so of them appear in more than 10 thesis titles.

In [32]:
wordcount_cutoff = 10
print(
    "Number of unique words: {}. Number of theses: {}. Number of words with more than {} occurrences: {}.".format(
        len(total_wordcounts),
        len(df),
        wordcount_cutoff,
        (total_wordcounts > wordcount_cutoff).sum(),
    )
)

Number of unique words: 117488. Number of theses: 200000. Number of words with more than 10 occurrences: 13123.


Thus I choose to restrict the graph to this subset of words, since it makes it computationally much lighter to handle, without causing much of a loss: Words that only occur in a handful of thesis titles probably wouldn't add much to our analysis anyway.

In [33]:
mainwords = allwords[total_wordcounts > wordcount_cutoff]

Note also that when making word co-occurrence graphs of texts, co-occurrences are often given more weight if the words are next to each rather than just in the same sentence. I don't make this distinction since the titles are pretty short anyway, and word orders in titles can be unusual, making proximity less relevant.

For the graph/network, I use the [NetworkX](https://networkx.github.io/) package. It represents graphs as adjacency lists, which suits our quite sparse co-occurrence graph well.

In [34]:
cooc_graph = nx.Graph()
cooc_graph.add_nodes_from(mainwords)
for words in df["Words"]:
    words = tuple(filter(lambda w: w in mainwords, words))
    for i in range(len(words)):
        wi = words[i]
        # The second loop starts from i+1 to avoid double-counting.
        for j in range(i + 1, len(words)):
            wj = words[j]
            if wi in cooc_graph[wj]:
                # This edge already exists, increment the weight
                cooc_graph[wi][wj]["weight"] += 1.0
            else:
                # This edge doesn't yet exist, create it.
                cooc_graph.add_edge(wi, wj, weight=1.0)

As we have it now, the edge weights of the graph just count the number of theses in which both words appear. This isn't quite what we want, since it's dominated by commonly occuring words. There's several different ways we could normalize the weights. Dividing by the total appearance counts of the words (diagonal of the adjacency matrix) would make the weight of the edge between nodes A and B represent a conditional probability (or rather a frequency) P(A|B) of word A appearing in a title given that word B appears. We could also make a bidirectional graph that has both P(A|B) and P(B|A) stored for each edge. Or we could do a number of more symmetric normalizations that don't have as concrete an interpretation. A choice that I've found works well for what I want to do with this graph later is normalizing the edge between A and B by average of the frequencies of A and B. This is symmetric (the graph is undirected) and roughly speaking gives a large weight for edges that connect two words that appear roughly equally frequently and often together.

In [35]:
for w1, w2 in cooc_graph.edges:
    avg_count = (total_wordcounts[w1] + total_wordcounts[w2]) / 2.0
    cooc_graph[w1][w2]["weight"] /= avg_count

Another handy function to have: One that, given a word, gets related words from the co-occurrence graph.

In [36]:
def get_related_words(w, n=5):
    """ For a given word w, get the n (by default n=5) words with
    heaviest edges connected to w in the co-occurrence graph.
    Returns a list of tuples (neighbour word, weight), sorted by weight.
    """
    if w not in cooc_graph:
        # This word is not in the co-occurrence network.
        return ()
    node = cooc_graph[w]
    neighbourlist = ((k, v["weight"]) for k, v in node.items())
    return sorted(neighbourlist, key=opr.itemgetter(1), reverse=True)[:n]

Finally, lets wrap in a function something we'll be doing repeatedly: Plotting the popularity of a word over time and printting out some often co-occuring words and example titles.

In [37]:
def output_word_summary(w, n_related=3, n_examples=3, plot=True):
    print("Word: {}".format(w))
    if n_related > 0:
        print("Related words:")
        [
            print(" {}: {}".format(*w))
            for w in get_related_words(w, n=n_related)
        ]
    if n_examples:
        print("Example titles:")
        [
            print(" {}".format(title))
            for title in get_example_titles(w, n=n_examples)
        ]
    if plot:
        fig, ax1 = plt.subplots(figsize=(5, 3))
        ax1.plot(wordcounts_by_year[w], color="black")
        title = "Number and proportion of theses\nwith '{}' in the title".format(
            w
        )
        ax1.set_title(title)
        ax1.set_ylabel("Absolute umber (black)".format(w))
        ax1.set_xlabel("Year")
        ax2 = ax1.twinx()
        ax2.plot(wordratios_by_year[w], color="darkred")
        ax2.set_ylabel("Per 1000 theses (red)".format(w))
        fig.tight_layout()

### Trends in individual words

With all that set-up out of the way, let's see if these titles hold some interesting. First off, what are the most commonly occuring words in thesis titles, and how does their popularity vary over time?

In [38]:
num_topwords = 3
for w in allwords[:num_topwords]:
    output_word_summary(w)

Word: study
Related words:
 case: 0.20922909880564602
 comparative: 0.11863266678896839
 development: 0.06386439517164626
Example titles:
 Ecological dynamics and human welfare : a case study of population, health and nutrition in Zimbabwe
 Theoretical study of binding energy and electronic energy levels in small clusters of metal atoms
 A case study of ESP for medical workplaces in Saudi Arabia from a needs analysis perspective


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Word: studies
Related words:
 structural: 0.0948864975257246
 synthesis: 0.05163344407530454
 synthetic: 0.05063073886552819
Example titles:
 Magneto-optical studies in semiconductors
 Permeation studies of hydrogen in nickel, molybdenum and M316 stainless steel : the influence of phase boundary processes
 Industrial boiler system corrosion inhibitors : Studies on the high temperature reactions and properties of aliphatic amines in water


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Word: development
Related words:
 system: 0.06889233531956575
 application: 0.06532399299474606
 study: 0.06386439517164626
Example titles:
 An exploratory study of research and development in construction in the developing countries of the middle East
 Development of CCD and EM-CCD technology for high resolution X-ray spectrometry
 The development of a methodology for automated sorting in the minerals industry


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Nothing too exciting here: Generic terms that appear in all kinds of titles. Both "study" and "studies" have clearly fallen into relative disuse since the 70s.

Much more interesting than the most popular words, are those whose popularity has been fleeting. One could get at this in a million different ways, but here's one that I found interesting. For each decade, find the top 3 words whose popularity that decade was the most disproportionately large compared to their popularity in the whole time window. In other words, rank words by number of occurences in a given decade, divided by total number of occurences. To not have this be dominated by words occuring in only a handful of theses, further restrict to words with at least 100 occurences in total. I'll also print for each word some of the commonly co-occurring words, to give some context.

In [39]:
num_topwords = 3
for start_year in range(1930, 2020, 10):
    end_year = start_year + 10
    print("{}s:".format(start_year))
    decade_top_words = (
        wordcounts_by_year[start_year:end_year].sum() / total_wordcounts
    )[total_wordcounts > 100].sort_values(ascending=False)
    for w in decade_top_words.index[:num_topwords]:
        # The w[0] picks just the word, and leaves out the weight.
        neighbours = [w[0] for w in get_related_words(w, n=3)]
        print("{:15}  {}".format(w, neighbours))
    print("-" * 79)

1930s:
cases            ['review', 'hundred', 'pathological']
substances       ['humic', 'similar', 'toxic']
tuberculosis     ['mycobacterium', 'pulmonary', 'bovine']
-------------------------------------------------------------------------------
1940s:
substances       ['humic', 'similar', 'toxic']
tuberculosis     ['mycobacterium', 'pulmonary', 'bovine']
constitution     ['discursive', 'alloys', 'certain']
-------------------------------------------------------------------------------
1950s:
substances       ['humic', 'similar', 'toxic']
microorganisms   ['methionine', 'photosynthetic', 'swimming']
helium           ['superfluid', 'temperatures', 'adsorbed']
-------------------------------------------------------------------------------
1960s:
spectra          ['raman', 'absorption', 'molecules']
geology          ['geochemistry', 'area', 'petrology']
testament        ['old', 'hebrews', 'epistle']
-------------------------------------------------------------------------------
1970s:
he

The 30s, 40s, and 50s look quite similar, with pretty generic terminology, especially science terminology trending high, but from that point on things get interesting. Plenty of particle physics in the 60s and 70s (a bubble chamber is a type of particle physics experiment and GeV/c, gigaelectronvolts per speed of light, is a a unit for momentum in particle physics), when new particles were constantle being discovered. The 80s, 90s and 2000s have a lot of computer science and communication technology words in the lead (ATM in most cases stands for asynchronous transfer mode), giving way to "mindfulness" and "resilience" in the last decade. They also highlight the relative lack of humanities and social sciences terminology in the list, but we'll come back that later.

Just for highlighting, here's a bit more detail on a few select words from the above list. The prospective rabbit holes are endless.

In [40]:
select_words = ["monoclonal", "globalisation", "graphene"]
for w in select_words:
    output_word_summary(w)

Word: monoclonal
Related words:
 antibodies: 0.5106382978723404
 antibody: 0.19148936170212766
 against: 0.04238921001926782
Example titles:
 The biology of immunoglobulin free light chains in kidney disease : a study of Monoclonal and Polyclonal light chains
 Physical and chemical studies of monoclonal antibodies to lysozyme
 Studies on pig kidney microvillar membrane proteins using monoclonal antibodies


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Word: globalisation
Related words:
 firmlevel: 0.03508771929824561
 hegemonic: 0.034782608695652174
 beijing: 0.032520325203252036
Example titles:
 Globalisation and the negotiation of identity in South Asian diasporic fiction in Britain
 Globalisation and ethnic integration in corporate Malaysia : the case of public-listed companies in the Klang Valley and Penang
 Globalisation and the labour market : an analysis of job stability and job security in Britain


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Word: graphene
Related words:
 epitaxial: 0.10909090909090909
 cvd: 0.06382978723404255
 dots: 0.06015037593984962
Example titles:
 Acoustoelectric transport in graphene
 Raman spectroscopy of graphene, its derivatives and graphene-based heterostructures
 The optical characterisation of graphene


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 12 points go to Biomed

Individual words like those above tell us stories of individual scientific discoveries and hot topics, but they have their limits if we want to study larger academic trends. However, one would naturally expect the co-occurrences of words in titles to follow patterns, where words related to fields and subfields would often appear together. Our next goal is to see if this indeed happens, and assuming it does (yes, it does), use it to
* identify what are the academic field distinctions that the co-occurence graph holds.
* analyse how the popularities of these different fields have varied.

So we want to find groups of words that typically appear together. This could be called clustering, but in the context of networks/graphs like our co-occurrence graph, it usually goes by the name of community structure. There's plenty of research done on developing algorithms for identifying communities. We use below one quite well-known one, the [Louvain algorithm](https://arxiv.org/abs/0803.0476), which is a heuristic algorithm based on optimizing the modularity of the graph, i.e. minimizing the weight of edges connecting different communities and maximizing the weight of edges internal to communities. I tried a few other methods as well, most notably label-propagation and stochastic block models, but at least with the parameters that I tried they seemed to produce communities that were either very small or did not match well my human intuition of which words I would expect to be related.

If you are running this notebook yourself, you can safely go make a cup of tea at this point. The community finding takes around  minutes on my laptop.

In [41]:
partition = community.best_partition(cooc_graph)

In [42]:
# louvain.best_partition returns a dictionary of {word: community_label}.
# Lets turn that into a DataFrame and extract the lists of words belonging to each community.
community_labels = pd.DataFrame(
    {"Label": tuple(partition.values())},
    index=partition.keys(),
    dtype="category",
)
communities = tuple(
    map(tuple, community_labels.groupby("Label").groups.values())
)

Let's take a look at what we found.

In [43]:
print("Number of communities: {}".format(len(communities)))
print(
    "Numbers of words in the communities: {}".format(
        [len(c) for c in communities]
    )
)
print("Some example words from each community:")
num_example_words = 40
[print(c[:num_example_words], end="\n\n") for c in communities];

Number of communities: 8
Numbers of words in the communities: [3357, 3621, 2760, 1818, 1558, 2, 3, 4]
Some example words from each community:
('study', 'development', 'analysis', 'case', 'social', 'between', 'reference', 'use', 'theory', 'management', 'approach', 'new', 'education', 'performance', 'aspects', 'learning', 'evaluation', 'impact', 'health', 'english', 'policy', 'british', 'towards', 'special', 'change', 'influence', 'children', 'practice', 'uk', 'england', 'assessment', 'process', 'political', 'comparative', 'language', 'relationship', 'through', 'information', 'critical', 'politics')

('studies', 'investigation', 'using', 'systems', 'control', 'modelling', 'design', 'system', 'synthesis', 'behaviour', 'properties', 'structure', 'model', 'application', 'applications', 'high', 'models', 'based', 'methods', 'dynamics', 'structural', 'techniques', 'processes', 'flow', 'data', 'networks', 'metal', 'experimental', 'energy', 'power', 'problems', 'reactions', 'structures', 'optic

So according to the Louvain algorithm, our co-occurrence graph has 19 communities, out of 5 are non-tiny. We'll just discard the 14 small ones as uninteresting (not every word needs to be in a community, recall that our graph only includes the ~20,000 most common words anyway), and focus on the 5 big ones.

Gladly, they make quite good sense to human intuition. The first one has a lot of social sciences vocabulary in it, we'll call it Social; the second one is clearly hard sciences, maths, and engineering or just Science for short; the second is medicine and biochemistry, aka Biomed; the fourth one clearly Humanities; and the fifth has a theme of ecology and geography, and we'll call it Eco/Geo.

At this point a note about stability of these communities is in order. I've been running the same community finding on graphs made with only subsets of the data set, leaving out some of the theses, and also tried including more or less words in the co-occurrence graph (remember we picked the arbitrary cutoff of only including words with at least 10 occurrences). The main communities fortunately appear quite stable under such perturbations. Including some what less words or less theses sometimes makes Humanities merge with Social sciences, but otherwise the communities stay roughly as they are. Going the other way, including more rare words in the graph may for instance sometimes cause particle physics to separate from the rest of Science. These kinds of granularity differences, of whether a field splits into a further subfields or merges with its academic neighbour, is quite natural, and doesn't in my view undermine the analysis in any significant way.

Next, what we would like to do is assign to each word and thesis title a score or label, that would tell us roughly which field(s) it belongs in. The naive way would be to just use to label each word with its respective community, and count how many humanities words appear in a given title, etc. This doesn't seem quite fair though. Clearly some words are in some sense more "deeply" in each community. For instance the word "theory" gets grouped into social sciences, but obviously it occurs in other contexts as well, unlike the word "policy", which is pretty dead give-away. To account for this effect, we'll give each word scores for how strongly they are connected to each community. This score starts out being 1 for the community the word belongs in and 0 for the others, but we add to it the total weight of edges connecting this word to words in a given community. So a word that is in the humanities community and co-occurs mostly with other humanities words gets a high humanities-score, whereas a word that co-occurs with words from several different communities will have significant scores for all of them. Finally, we'll normalize the scores by the total sum within a field, so for instance the Hum/Soc score of each word will be divided by the sum of Hum/Soc scores of all words. This accounts for the fact that some fields include more words, and these words may be more strongly connected in our co-occurrence graph. We'll also multiply the resulting scores by 10,000, just to produce more human-readable numbers.

First off, use some key words in each community to anchor the names of the fields.

In [44]:
field_keywords = {
    "education": "Social",
    "cell": "Biomed",
    "magnetism": "Science",
    "ecology": "Eco/Geo",
    "philosophy": "Humanities",
}

First give each word a score of 1.0 for the community it belongs in according to the Louvain classification. Drop all the small communities we don't care about, and name the columns of the DataFrame with the names given above.

In [45]:
wordscores = pd.get_dummies(community_labels).astype(np.float_)

In [46]:
for word, field in field_keywords.items():
    current_label = wordscores.loc[word, :].idxmax()
    wordscores.rename(columns={current_label: field}, inplace=True)
for c in wordscores.columns:
    if not c in field_keywords.values():
        wordscores.drop(c, axis=1, inplace=True)
fieldnames = wordscores.columns

Then add to the scores the weights of the edges connecting each word to words of different communities. This is easy to do with a matrix product with the adjacency matrix of the graph.

In [47]:
# So to get the scores we want, we need to take the matrix product of the
# adjacency matrix of the co-occurrence graph with the current score matrix,
# wordscores.values. The graph is way too big to build the whole adjacency
# matrix as a dense matrix, but luckily NetworkX has us covered, with the
# possibility of creating a scipy sparse matrix instead. However, the pandas
# dot function for matrix products insists in converting everything to a dense
# matrix to do the product, so we'll have to manually work with the
# scipy.sparse matrix instead of a DataFrame. It's not too bad though.
adjacency_matrix = nx.to_scipy_sparse_matrix(
    cooc_graph, nodelist=wordscores.index, format="csr"
)
for c in fieldnames:
    column_vec = wordscores[c].to_numpy()
    scores = adjacency_matrix.dot(column_vec)
    scores_series = pd.Series(scores, index=wordscores.index)
    wordscores[c] += scores_series
del(adjacency_matrix)  # Release the memory

Finally, normalize the scores, and give all the words we did not include in our graph (the ones with fewer than 10 occurrences) a score of 0.0 for everything.

In [48]:
wordscores *= 10000 / wordscores.sum(axis=0)

In [49]:
wordscores = wordscores.reindex(allwords, fill_value=0.0)

To check that the scoring system makes sense, let's check the scores for the top words for each field.

In [50]:
num_topwords = 6
for c in fieldnames:
    print("\nWords with highest scores for {}:".format(c))
    print(wordscores.sort_values(c, ascending=False)[:num_topwords])


Words with highest scores for Social:
               Social   Science    Biomed  Humanities   Eco/Geo
education    7.487827  0.528913  0.463062    1.533165  0.535413
case         7.363024  0.946124  0.639468    1.121846  1.477142
school       7.188815  0.559221  0.580077    1.027660  0.469706
students     7.107860  0.530365  0.476748    0.744718  0.353932
experiences  6.991886  0.440224  1.060326    0.904114  0.330859
teachers     6.859119  0.404379  0.380870    0.745388  0.390275

Words with highest scores for Science:
                Social   Science    Biomed  Humanities   Eco/Geo
using         2.052565  6.305451  2.320687    0.423059  1.365946
high          1.360270  6.203214  1.272575    0.292053  0.944587
flow          0.668661  5.803994  1.122301    0.141784  1.320929
detector      0.228792  5.803482  0.360734    0.212696  0.179432
optical       0.442634  5.746755  0.590107    0.136178  0.337319
spectroscopy  0.253565  5.665409  0.817261    0.155537  0.416699

Words with highes

There's a few funny ones, like "L" being the top word in Eco/Geo because it often stands for Carl Linnaeus, the founder of modern taxonomy, in scientific names of species, but overall the scores seem pretty reasonable.

Next up, computing the scores for each thesis title, by just taking the average of the scores of individual words in the title. This takes a minute or so.

In [51]:
for c in fieldnames:
    print("Computing title scores for {}.".format(c))
    # The python sum function is significantly faster here than a sub-DataFrame.sum().
    df[c] = df["Words"].apply(
        lambda ws: 0.0
        if not ws
        else sum(wordscores.loc[w, c] for w in ws) / len(ws)
    )

Computing title scores for Social.
Computing title scores for Science.
Computing title scores for Biomed.
Computing title scores for Humanities.
Computing title scores for Eco/Geo.


## Trends in academic fields

Now let's see how the scores are distributed.

In [52]:
num_fields = len(fieldnames)
fig, axes = plt.subplot(nrows=num_fields, ncols=num_fields, figsize=(7, 7))
for ix, cx in enumerate(fieldnames):
    for iy, cy in enumerate(fieldnames):
        df[cx, cy]

AttributeError: Unknown property nrows

In [57]:
for c in fieldnames:
    plt.figure()
    sns.kdeplot(df[c])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [88]:
bins = 70
maxscore = 5


def hist2d(x, y, color, **kwargs):
    cmap = sns.light_palette(color, as_cmap=True)
    plt.hist2d(x, y, bins=(bins, bins), cmap=cmap, **kwargs)


def hist(x, color, **kwargs):
    plt.hist(x, bins=bins, **kwargs)

g = sns.PairGrid(df[fieldnames], height=1.6)
g.map_offdiag(hist2d)
g.map_diag(hist)
g.set(ylim=(0, maxscore), xlim=(0, maxscore))

/usr/lib/python3/dist-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [55]:
for c in fieldnames:
    plt.figure()
    df[c].hist(bins=200)
    plt.title(c)
    plt.xlim(0.0, 7.0)
    if c == "Eco/Geo":
        plt.ylim(0, 25000)
    elif c == "Humanities":
        plt.ylim(0, 35000)
    else:
        plt.ylim(0, 13000)
    plt.ylabel("Number of theses")
    plt.xlabel("Score for {}".format(c))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

These are histograms of how many theses get which score for the various fields. Note that the first three have the same scale for the vertical axis, but Eco/Geo does not.

The first peak in each one of these is not the most interesting part. Since we smudged the boundaries of fields by adding neighbouring edge weights when scoring words, most words have a non-zero score for all the fields. The first peaks then represent a sort of baseline value, of what's a usual score for a thesis that isn't really in the given field. The fact that the positions of the first peaks for all four fields roughly line up is an indicator that our normalization of the scores was somewhat successful, and there's meaning to comparing scores across fields.

The tails, from score 1.5 or so right-wards, are where the action is. Eco/Geo has a pretty smoothly declining distribution and small values overall, since it is by far the least popular of the fields, presumably because it is the most specific. Other than that, the main difference between fields here is that whereas Science and Hum/Soc have a big solid bulge around scores 2.0 - 4.0, Biomed in contrast has a much thinnes but longer tail reaching all the way to scores larger than 5.0. Roughly speaking this means that a thesis either is or isn't Biomed, with less middle ground compared to the Hum/Soc and Science scores that leave more ambiguity.

For these field labels to be good labels, they shouldn't have much correlation between them, at least not positive correlation, so let's check that.

In [ ]:
corrmat = df[wordscores.columns].corr()
plt.matshow(corrmat)
print(corrmat)

Indeed, we are doing well here. Having a high Hum/Soc score makes it unlikely to also score highly on Biomed or Science, but otherwise the different labels are pretty independent. This fits at least my intuitive expectations, although I'm a little surprised that among the more Nature-oriented fields of Biomed, Science and Eco/Geo the little correlation that exists is negative: I would have guessed topics like chemistry and population biology to cause some mixing between them.

Being pretty happy with our scoring method, lets see what trends we can detect over time and institutions.

First off, here are the mean scores of theses published in each year, over time.

In [ ]:
meanscores_by_year = df.groupby(["Date"]).mean().fillna(0.0)
meanscores_by_year.plot()

There's quite a lot going on here. Science and Hum/Soc seem to be at odds, with Hum/Soc having a V-shaped development of popularity first declining until around 1970 and then slowly growing back again to its former glory, and Science following roughly a reverse of this. Especially notable is the huge bump in Science scores in the 60s, before the steady decline sets in. Biomed has the steadiest trend, with slow growth overall, whereas Eco/Geo remains pretty stable until the early 80s, after which it sets into a steady decline, that seems to have leved off lately.

To put it more poetically, the one feature to highlight is that the latter half of the last century was heavily dominated by hard sciences and engineering, at the expense of humanities and social sciences, which have then recovered and regained their past status. It would be very interesting to try to compare this to other societal and academic trends, such as gender distributions of PhD graduates, changes in academic funding in the UK, and data on students at earlier stages in their education, such as the popularity of different undergraduate majors and changes in primary and secondary education curricula. For the moment, though, this remains work to be done.

Finally, we can observe academic trends for individual institutions. For instance, here are the same plots as above, but specific to some of the golden triangle universities.

In [ ]:
schools = [
    "University of Oxford",
    "University of Cambridge",
    "Imperial College London",
    "London School of Economics and Political Science (University of London)",
]
meanscores_by_yearandinst = (
    df.groupby(["Institution", "Date"])
    .mean()
    .unstack("Institution")
    .fillna(0.0)
    .reorder_levels([1, 0], axis=1)
)
ylims = (-0.05, 4.05)
for school in schools:
    plt.figure()
    lines = plt.plot(meanscores_by_yearandinst[school])
    plt.title(school)
    plt.ylim(*ylims)
    plt.legend(lines, meanscores_by_yearandinst[school].columns)
    plt.xlabel("Year")
    plt.ylabel("Mean field score of thesis titles")

Hardly surprisingly, London School of Economics is heavily leaning towards humanities and social sciences, and Imperial lives up to its reputation of being a science school first and foremost. The above also provides evidence for the wide-spread view that of the Oxbridge two, Cambridge is the more science heavy one, whereas Oxford leans towards Hum/Soc, although the difference isn't huge, at least not by our metric.

In [ ]:
g.edges[:100]

In [ ]:
# TODO
# - Abnormal titles with far-apart words (inverse weights, all_pairs_dijkstra_path_length).
# - Do a visualization of the graph?
# - We vs I
# - Can I find correlations between words in titles at institutions, and their university rankings? Suggestion: https://www.leidenranking.com/ranking/2019/chart
# - Possible hypotheses to test: Humanities words have less peaked popularity profiles.
# - Credit feedback
# - Mention more robust comparison of community finding algorithms could be done.

Finally, if you dig into this dataset yourself and find interesting features I've missed, please let me know: markus@mhauru.org

In [ ]:
word = "finite"
print(wordscores.loc[word, :])
print()
[print(w) for w in get_related_words(word, 5)]
print()
[print(w) for w in get_example_titles(word, 5)];